# Práctica 1 - Cuadraturas adaptativas

Gil Molina, Ana, Zhong, Wenya

## Ejercicio 1 
Implementar en Python un programa **_SC_** que integre numéricamente una función $f(x)$ sobre un intervalo $[a,b]$ basado en la _regla de Simpson compuesta_ y en el que el usuario especifique el número $m$ de subintervalos.


Parámetros de entrada: 
   * _f_ : expresión de la función $f(x)$ a integrar 
   * _rango_ : intervalo $[a,b]$ sobre el que se quiere aproximar la integal 
   * _m_: el número  de subintervalos.
   
Salida: 
   * I : el valor aproximado de la integral
 


_Recordatorio_ : Fórmula de _Simpson compuesto_ con _m_ intervalos
$$
F_{SC}^{(m)}=\frac{h}{3} \left[f(x_0)+f(x_2m)+2\sum_{j=1}^{m-1}f(x_{2j})+4\sum_{j=1}^{m}f(x_{2j-1}) \right],\text{ donde } h=\frac{b-a}{2m}.
$$


### Pseudocódigo del algoritmo
 
>   Input: f, [a,b], m 
>
>      h = (b - a)/2m
>
>      I0=f(a)+f(b)
>
>      I1=0 # suma de f(xi), i impar
>
>       I2=0 # suma de f(xi), i par
>
>       for k = 1..(2m-1) do
>
>           x=a+k*h
>
>           Si k es par entonces 
>
>              I2=I2+f(x)
>
>              sino I1=I1+f(x)
>
>           end si 
>
>       end do 
>
>       I=h*(I0+2*I2+4*I1)/3
>
>   Output I
>
>   Fin
 


### Programa SC

In [3]:
def SC(f, a, b, m):
    h = (b-a)/(2*m)
    I0 = f(a)+f(b)
    I1 = 0
    I2 = 0
    for k in range(1,2*m):
        x = a+k*h
        if k%2 == 0:
            I2 = I2 + f(x)
        else:
            I1 = I1 + f(x)
    I = h*(I0+2*I2+4*I1)/3
    return I

### Ejemplos

In [10]:
import numpy as np
import sys

def f(x):
    return np.exp(2*x)

SC(f,0,1,2)

3.195605093333459

## Ejercicio 2 (7 puntos)
Mejorar la aproximación de la integral del apartado anterior mediante un procedimiento recursivo **_SC_ADAPT_** que realice una cuadratura adaptativa basada en la regla de Simpson compuesta. Podéis elegir cualquiera de las dos opciones explicadas en clase.
 
Parámetros de entrada: 
   * _f_ : eexpresión de la función $f(x)$ a integrar 
   * _rango_ : intervalo $[a,b]$ sobre el que se quiere aproximar la integral 
   * _tol_ :  la tolerancia
   
  
Opciones: 
   * _ini_intervalos_ :   el número de intervalos de la partición inicial (valor por defecto: ini_intervalos=2)
   * _max_intervalos_ :  el número máximo de intervalos permitido (valor por defecto: max_intervalos=1000)
 
 
Salida: 
   * _I_ : valor aproximado de la integral 
   * _n_: número de subintervalos usados
   
## <font color='red'> **OPCIONAL**</font> (3 puntos)

Añadir una nueva opción _metodo_ que nos permita escoger entre las dos opciones vistas en clase:
* Si _metodo_ = 1 : Aplicamos la opción de dividir los intervalos a la mitad. Si SC(f,a,b) no cumple con la tolerancia, dividimos el intervalo en 2, calculando SC(f,a,(a+b)/2) y SC(f,(a+b)/2,b)
* Si _metodo_ = 2: Aplicamos la subdivisión adaptativa. Si SC(f,a,b) no cumple la tolerancia, tenemos que encontrar un c, con a<c<b, tal que SC(f,a,c) cumpla la tolerancia. Calculamos SC(f,a,c) y SC(f,c,b)


### Pseudocódigo del algoritmo


>   Input: f, [a,b], tol, opciones (max_intervalos,ini_intervalos,metodo,contador,devolver) (opcional)
>
>        Lectura, comprobación y valores por defecto de las opciones
>
>        si max_intervalos <= 0 entonces
>            Error
>
>        si ini_intervalos es 1 entonces
>            S = SC(f,a,b,1)
>            S_ = SC(f,a,b,2)
>            E = (S_ - S)/15
>
>            si abs(E) <= tol*(b-a) entonces
>                I = S_
>            sino
>                contador += 1
>                max_intervalos -= 1
>
>                si metodo es 1 entonces
>                    I = SC_ADAPT(f,a,(a+b)/2) + SC_ADAPT(f,(a+b)/2,b)
>
>                si metodo es 2 entonces
>                    Buscar a < c < b para que se cumpla la tolerancia
>                    I = SC_ADAPT(f,a,c) + SC_ADAPT(f,c,b) 
>
>        si ini_intervalos no es 1 entonces
>            h=(b-a)/ini_intervalos
>            I = 0
>
>            for i = 1,..., ini_intervalos+1:
>                contador += 1
>                max_intervalos -= 1
>                I += SC_ADAPT(f,a+(i-1)h,a+ih)
>
>    Output: I, contador
>
>    Fin

### Programa SC_ADAPT


In [4]:
def SC_ADAPT(f, a, b, tol, **kwargs):
    
    # Lectura de opciones
    max_intervalos=kwargs.get("max_intervalos")
    ini_intervalos=kwargs.get("ini_intervalos")
    metodo=kwargs.get("metodo")
    contador=kwargs.get("contador")
    devolver=kwargs.get("devolver")
    
    # Comprobación sobre las opciones introducidas
    opc=[k for k in kwargs]
    opciones=["max_intervalos","ini_intervalos","metodo","contador","devolver"]
    
    # Comprobación sobre las opciones introducidas
    opc_extra=[i for i in opc if not i in opciones]
    if len(opc_extra)>0:
        print("Advertencia: no se tendrán en cuenta las opciones:{0}".format(opc_extra))
    
    # Asignación de valores por defecto
    if not "max_intervalos" in opc:
        max_intervalos=1000
    if not "ini_intervalos" in opc:
        ini_intervalos=2 
    if not "metodo" in opc:
        metodo=1
    if not "contador" in opc:
        contador=1
    if not "devolver" in opc:
        devolver=0
        
    if max_intervalos <= 0:
        raise SystemError('Error: número máximo de intervalos agotado')
    
    if ini_intervalos == 1:
        S = SC(f,a,b,1)
        S_ = SC(f,a,b,2)
        E = (S_ - S)/15
        
        if abs(E) <= tol*(b-a):
            I = S_
        else:
            contador += 1
            max_intervalos -= 1

            if metodo == 1:
                I1, contador, max_intervalos = SC_ADAPT(f,a,(a+b)/2,tol,max_intervalos=max_intervalos,contador=contador,ini_intervalos=1,devolver=1)
                I2, contador, max_intervalos = SC_ADAPT(f,(a+b)/2,b,tol,max_intervalos=max_intervalos,contador=contador,ini_intervalos=1,devolver=1)
                I = I1 + I2
                
            elif metodo == 2:
                c = b
                while abs(E) > tol*(c-a):
                    c = (a+c)/2
                    S = SC(f,a,c,1)
                    S_ = SC(f,a,c,2)
                    E = (S_ - S)/15
                I1, contador, max_intervalos = SC_ADAPT(f,a,c,tol,max_intervalos=max_intervalos,contador=contador,metodo=2,ini_intervalos=1,devolver=1)
                I2, contador, max_intervalos = SC_ADAPT(f,c,b,tol,max_intervalos=max_intervalos,contador=contador,metodo=2,ini_intervalos=1,devolver=1)
                I = I1 + I2

    else:
        h=(b-a)/ini_intervalos
        I = 0
        for i in range(1,ini_intervalos+1):
            contador += 1
            max_intervalos -= 1
            I_sub, contador, max_intervalos = SC_ADAPT(f,a+(i-1)*h,a+i*h,tol,max_intervalos=max_intervalos,contador=contador,metodo=metodo,ini_intervalos=1,devolver=1)
            I += I_sub
        
    if devolver == 0:
        return I, contador
    
    elif devolver == 1:
        return I, contador, max_intervalos       

### Ejemplos

In [24]:
import numpy as np
import sys

def f(x):
    return np.exp(2*x)

print(SC_ADAPT(f,0,1,0.00000000001))

print(SC_ADAPT(f,0,1,0.00000001))

print(SC_ADAPT(f,0,1,0.00000001,metodo=2))

print(SC_ADAPT(f,0,1,0.00000001,ini_intervalos=3))

print(SC_ADAPT(f,0,1,0.00000001,max_intervalos=28))

SC_ADAPT(f,0,1,0.00000001,max_intervalos=27)

(3.194528049469457, 129)
(3.19452805267917, 28)
(3.194528053097221, 26)
(3.1945280528084052, 25)
(3.19452805267917, 28)


SystemError: Error: número máximo de intervalos agotado

## Ejercicio 3
Probar vuestros programas para calcular las siguientes integrales, modificando el número de intervalos para _SC_ , ó la cota de error para _SC_ADAPT_ , y comparando vuestra aproximación con el valor exacto. Comparad los tiempos de cálculo para cada caso según se explicó en *Practica0_JupiterNotebook.ipynb*.  

   * $ \displaystyle \int_0^1 e^{2x}=3.194528050 $
   
   * $ \displaystyle \int_2^{10} \sin{4x^2}=-0.0536589163 $
   
   * $ \displaystyle \int_0^1 \ln(3x+1) +\cos^2(-2x)=1.253792170$
   
   * $ \displaystyle \int_0^{0.2} e^{-x^2}=0.1973650309$
   
   * $ \displaystyle \int_1^{1.5} x^3\ln(x)=0.259260532$
   
   
   
   
   


In [93]:
import numpy as np
import sys
import time

def f1(x):
    return np.exp(2*x)

v_exacto1 = 3.194528050

for i in [3,4,10]:
    time_start = time.perf_counter()
    I = SC(f1,0,1,i)
    t = time.perf_counter() - time_start
    print("Con",i,"intervalos:",I)
    print("Diferencia con valor exacto:",abs(I-v_exacto1))
    print("Tiempo de cálculo:",t*1000, "milisegundos")
    print()

# Cuantos más intervalos, observamos que se aproxima más al valor exacto 3.194528050

for tol in [0.1,0.00001,0.0000000001]:
    time_start = time.perf_counter()
    I = SC_ADAPT(f1,0,1,tol)[0]
    t = time.perf_counter() - time_start
    print("Con tolerancia",tol,":",I)
    print("Diferencia con valor exacto:",abs(I-v_exacto1))
    print("Tiempo de cálculo:",t*1000, "milisegundos")
    print()
    
# Cuanto menor tolerancia, observamos que se aproxima más al valor exacto 3.194528050

Con 3 intervalos: 3.1947442881370556
Diferencia con valor exacto: 0.0002162381370554023
Tiempo de cálculo: 0.054799998906673864 milisegundos

Con 4 intervalos: 3.1945968627082117
Diferencia con valor exacto: 6.881270821157415e-05
Tiempo de cálculo: 0.0411000000895001 milisegundos

Con 10 intervalos: 3.194529822092564
Diferencia con valor exacto: 1.7720925638009533e-06
Tiempo de cálculo: 0.06100000064179767 milisegundos

Con tolerancia 0.1 : 3.1945968627082117
Diferencia con valor exacto: 6.881270821157415e-05
Tiempo de cálculo: 0.1159999992523808 milisegundos

Con tolerancia 1e-05 : 3.194532374274869
Diferencia con valor exacto: 4.324274868849898e-06
Tiempo de cálculo: 0.1861000000644708 milisegundos

Con tolerancia 1e-10 : 3.194528049506038
Diferencia con valor exacto: 4.93962204473064e-10
Tiempo de cálculo: 4.364000000350643 milisegundos



In [97]:
import numpy as np
import sys
import time

def f2(x):
    return np.sin(4*x**2)

v_exacto2 = -0.0536589163

for i in [3,10,90,300]:
    time_start = time.perf_counter()
    I = SC(f2,2,10,i)
    t = time.perf_counter() - time_start
    print("Con",i,"intervalos:",I)
    print("Diferencia con valor exacto:",abs(I-v_exacto2))
    print("Tiempo de cálculo:",t*1000, "milisegundos")
    print()

# Necesita muchos intervalos para acercarse al valor exacto -0.0536589163

for tol in [0.1,0.001,0.00001]:
    time_start = time.perf_counter()
    I = SC_ADAPT(f2,2,10,tol)[0]
    t = time.perf_counter() - time_start
    print("Con tolerancia",tol,":",I)
    print("Diferencia con valor exacto:",abs(I-v_exacto2))
    print("Tiempo de cálculo:",t*1000, "milisegundos")
    print()

Con 3 intervalos: -1.9923686572997428
Diferencia con valor exacto: 1.9387097409997427
Tiempo de cálculo: 0.08139999954437371 milisegundos

Con 10 intervalos: -0.8607154814584711
Diferencia con valor exacto: 0.8070565651584711
Tiempo de cálculo: 0.14899999951012433 milisegundos

Con 90 intervalos: 0.052237643821282875
Diferencia con valor exacto: 0.10589656012128287
Tiempo de cálculo: 0.7604000002174871 milisegundos

Con 300 intervalos: -0.0536046906995575
Diferencia con valor exacto: 5.42256004424993e-05
Tiempo de cálculo: 1.9207000004826114 milisegundos

Con tolerancia 0.1 : -2.0433161632263133
Diferencia con valor exacto: 1.9896572469263132
Tiempo de cálculo: 0.09249999857274815 milisegundos

Con tolerancia 0.001 : -0.18921961518538866
Diferencia con valor exacto: 0.13556069888538866
Tiempo de cálculo: 3.328799999508192 milisegundos

Con tolerancia 1e-05 : -0.05365852898760223
Diferencia con valor exacto: 3.873123977649051e-07
Tiempo de cálculo: 24.51389999987441 milisegundos



In [101]:
import numpy as np
import sys
import time

def f3(x):
    return np.log(3*x+1) + np.cos(-2*x)**2

v_exacto3 = 1.253792170

for i in [3,4,10]:
    time_start = time.perf_counter()
    I = SC(f3,0,1,i)
    t = time.perf_counter() - time_start
    print("Con",i,"intervalos:",I)
    print("Diferencia con valor exacto:",abs(I-v_exacto3))
    print("Tiempo de cálculo:",t*1000, "milisegundos")
    print()
    
for tol in [0.1,0.0001,0.0000000001]:
    time_start = time.perf_counter()
    I = SC_ADAPT(f3,0,1,tol)[0]
    t = time.perf_counter() - time_start
    print("Con tolerancia",tol,":",I)
    print("Diferencia con valor exacto:",abs(I-v_exacto3))
    print("Tiempo de cálculo:",t*1000, "milisegundos")
    print()

Con 3 intervalos: 1.2535061426212415
Diferencia con valor exacto: 0.00028602737875860207
Tiempo de cálculo: 0.1283999990846496 milisegundos

Con 4 intervalos: 1.2536969775564686
Diferencia con valor exacto: 9.519244353151102e-05
Tiempo de cálculo: 0.0901000003068475 milisegundos

Con 10 intervalos: 1.2537895353631978
Diferencia con valor exacto: 2.6346368022789335e-06
Tiempo de cálculo: 0.18600000112201087 milisegundos

Con tolerancia 0.1 : 1.2536969775564684
Diferencia con valor exacto: 9.519244353173306e-05
Tiempo de cálculo: 0.1871999993454665 milisegundos

Con tolerancia 0.0001 : 1.2537701442285252
Diferencia con valor exacto: 2.2025771474920575e-05
Tiempo de cálculo: 0.26729999990493525 milisegundos

Con tolerancia 1e-10 : 1.2537921695642296
Diferencia con valor exacto: 4.3577053077115124e-10
Tiempo de cálculo: 8.636399999886635 milisegundos



In [104]:
import numpy as np
import sys
import time

def f4(x):
    return np.exp(-x**2)

v_exacto4 = 0.1973650309

for i in [2,4]:
    time_start = time.perf_counter()
    I = SC(f4,0,0.2,i)
    t = time.perf_counter() - time_start
    print("Con",i,"intervalos:",I)
    print("Diferencia con valor exacto:",abs(I-v_exacto4))
    print("Tiempo de cálculo:",t*1000, "milisegundos")
    print()
    
for tol in [0.1,0.9,10,1000000000]:
    time_start = time.perf_counter()
    I = SC_ADAPT(f4,0,0.2,tol)[0]
    t = time.perf_counter() - time_start
    print("Con tolerancia",tol,":",I)
    print("Diferencia con valor exacto:",abs(I-v_exacto4))
    print("Tiempo de cálculo:",t*1000, "milisegundos")
    print()

# Incluso con tolerancias muy altas se aproxima al valor exacto

Con 2 intervalos: 0.19736510908356408
Diferencia con valor exacto: 7.818356409505789e-08
Tiempo de cálculo: 0.045900000259280205 milisegundos

Con 4 intervalos: 0.197365035800569
Diferencia con valor exacto: 4.90056900614988e-09
Tiempo de cálculo: 0.056400000175926834 milisegundos

Con tolerancia 0.1 : 0.19736503580056905
Diferencia con valor exacto: 4.900569061661031e-09
Tiempo de cálculo: 0.09189999946102034 milisegundos

Con tolerancia 0.9 : 0.19736503580056905
Diferencia con valor exacto: 4.900569061661031e-09
Tiempo de cálculo: 0.09300000056100544 milisegundos

Con tolerancia 10 : 0.19736503580056905
Diferencia con valor exacto: 4.900569061661031e-09
Tiempo de cálculo: 0.0836000017443439 milisegundos

Con tolerancia 1000000000 : 0.19736503580056905
Diferencia con valor exacto: 4.900569061661031e-09
Tiempo de cálculo: 0.18869999985327013 milisegundos



In [25]:
import numpy as np
import sys
import time

def f5(x):
    return x**3*np.log(x)

v_exacto5 = 0.259260532

for i in [3,1]:
    time_start = time.perf_counter()
    I = SC(f5,1,1.5,i)
    t = time.perf_counter() - time_start
    print("Con",i,"intervalos:",I)
    print("Diferencia con valor exacto:",abs(I-v_exacto5))
    print("Tiempo de cálculo:",t*1000, "milisegundos")
    print()

# Tan solo con un intervalo se aproxima al valor exacto

for tol in [0.1,10,10000000]:
    time_start = time.perf_counter()
    I = SC_ADAPT(f5,1,1.5,tol)[0]
    t = time.perf_counter() - time_start
    print("Con tolerancia",tol,":",I)
    print("Diferencia con valor exacto:",abs(I-v_exacto5))
    print("Tiempo de cálculo:",t*1000, "milisegundos")
    print()

# Incluso con tolerancias muy altas se aproxima al valor exacto

Con 3 intervalos: 0.25926117850500957
Diferencia con valor exacto: 6.465050095827429e-07
Tiempo de cálculo: 0.08109999998850981 milisegundos

Con 1 intervalos: 0.2593128112089432
Diferencia con valor exacto: 5.2279208943184496e-05
Tiempo de cálculo: 0.05050000072515104 milisegundos

Con tolerancia 0.1 : 0.2592607335486735
Diferencia con valor exacto: 2.0154867352850658e-07
Tiempo de cálculo: 0.14109999938227702 milisegundos

Con tolerancia 10 : 0.2592607335486735
Diferencia con valor exacto: 2.0154867352850658e-07
Tiempo de cálculo: 0.10610000026645139 milisegundos

Con tolerancia 10000000 : 0.2592607335486735
Diferencia con valor exacto: 2.0154867352850658e-07
Tiempo de cálculo: 0.10259999999107094 milisegundos

